# Try out framework with simulated data

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display

from q2_ritme.process_data import load_n_split_data
from q2_ritme.tune_models import run_all_trials
from q2_ritme.evaluate_models import get_best_model, get_best_data_processing
from q2_ritme.feature_space.transform_features import transform_features
from q2_ritme.config import (
    HOST_ID,
    TARGET,
    SEED_DATA,
    SEED_MODEL,
    MLFLOW_TRACKING_URI,
)
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np

# 30.437 is avg. number of days per month
DAYS_PER_MONTH = 30.437
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load and split data

In [2]:
train_val, test = load_n_split_data()

Train: (75, 22), Test: (25, 22)


## Run all experiments

In [6]:
result_dic = run_all_trials(
    train_val, TARGET, HOST_ID, SEED_DATA, SEED_MODEL, MLFLOW_TRACKING_URI
)
result_dic

Ray tune training of: xgb...


2023-12-15 09:46:11,657	INFO worker.py:1625 -- Started a local Ray instance.
2023-12-15 09:46:13,086	INFO tune.py:218 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
/Users/anjaadamov/opt/anaconda3/envs/time/lib/python3.8/site-packages/ray/tune/experiment/experiment.py:170: UserWarning: The `local_dir` argument of `Experiment is deprecated. Use `storage_path` or set the `TUNE_RESULT_DIR` environment variable instead.
  warnings.warn(


(train_xgb pid=39799) [09:46:21] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
(train_xgb pid=39799) Parameters: { "data_alr_denom_idx" } are not used.
(train_xgb pid=39799) 


(train_xgb pid=39799) Train: (48, 22), Test: (27, 22)


(train_xgb pid=39827) Parameters: { "data_alr_denom_idx", "data_transform" } are not used.
(train_xgb pid=39827) 
(train_xgb pid=39827) 


...

(train_rf pid=40295) Train: (48, 22), Test: (27, 22)
(train_rf pid=40305) Train: (48, 22), Test: (27, 22)
(train_rf pid=40315) Train: (48, 21), Test: (27, 21)
(train_rf pid=40327) Train: (48, 21), Test: (27, 21)
(train_rf pid=40336) Train: (48, 22), Test: (27, 22)
(train_rf pid=40347) Train: (48, 22), Test: (27, 22)
(train_rf pid=40356) Train: (48, 21), Test: (27, 21)


Trial name,date,done,experiment_tag,hostname,iterations_since_restore,model_path,node_ip,pid,rmse_train,rmse_val,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
train_rf_4817a_00000,2023-12-15_09-52-42,True,"0_bootstrap=True,data_alr_denom_idx=11,data_transform=None,max_depth=8,max_features=log2,min_impurity_decrease=0.0001,min_samples_leaf=0.0001,min_samples_split=0.0010,n_estimators=767",fsbt142-114,1,"/Users/anjaadamov/Documents/projects/14_LM1/q2-time/experiments/ray_results/rf/train_rf_4817a_00000_0_bootstrap=True,data_alr_denom_idx=11,data_transform=None,max_depth=8,max_features=log2,min_impurity_decreas_2023-12-15_09-52-32/model.pkl",127.0.0.1,40295,82.0075,261.983,1.46819,1.46819,1.46819,1702630362,1,4817a_00000
train_rf_4817a_00001,2023-12-15_09-52-49,True,"1_bootstrap=False,data_alr_denom_idx=17,data_transform=clr,max_depth=27,max_features=log2,min_impurity_decrease=0.0001,min_samples_leaf=0.0010,min_samples_split=0.0100,n_estimators=753",fsbt142-114,1,"/Users/anjaadamov/Documents/projects/14_LM1/q2-time/experiments/ray_results/rf/train_rf_4817a_00001_1_bootstrap=False,data_alr_denom_idx=17,data_transform=clr,max_depth=27,max_features=log2,min_impurity_decrea_2023-12-15_09-52-40/model.pkl",127.0.0.1,40305,76.2636,277.273,1.3568,1.3568,1.3568,1702630369,1,4817a_00001
train_rf_4817a_00002,2023-12-15_09-52-55,True,"2_bootstrap=True,data_alr_denom_idx=8,data_transform=ilr,max_depth=15,max_features=0.1000,min_impurity_decrease=0.0100,min_samples_leaf=0.0001,min_samples_split=0.0100,n_estimators=304",fsbt142-114,1,"/Users/anjaadamov/Documents/projects/14_LM1/q2-time/experiments/ray_results/rf/train_rf_4817a_00002_2_bootstrap=True,data_alr_denom_idx=8,data_transform=ilr,max_depth=15,max_features=0.1000,min_impurity_decrea_2023-12-15_09-52-48/model.pkl",127.0.0.1,40315,81.9594,266.677,0.513516,0.513516,0.513516,1702630375,1,4817a_00002
train_rf_4817a_00003,2023-12-15_09-53-03,True,"3_bootstrap=True,data_alr_denom_idx=3,data_transform=alr,max_depth=25,max_features=log2,min_impurity_decrease=0.0100,min_samples_leaf=0.0000,min_samples_split=0.1000,n_estimators=517",fsbt142-114,1,"/Users/anjaadamov/Documents/projects/14_LM1/q2-time/experiments/ray_results/rf/train_rf_4817a_00003_3_bootstrap=True,data_alr_denom_idx=3,data_transform=alr,max_depth=25,max_features=log2,min_impurity_decrease_2023-12-15_09-52-55/model.pkl",127.0.0.1,40327,81.3416,249.476,0.947438,0.947438,0.947438,1702630383,1,4817a_00003
train_rf_4817a_00004,2023-12-15_09-53-11,True,"4_bootstrap=False,data_alr_denom_idx=16,data_transform=None,max_depth=16,max_features=0.1000,min_impurity_decrease=0.0001,min_samples_leaf=0.0001,min_samples_split=0.1000,n_estimators=772",fsbt142-114,1,"/Users/anjaadamov/Documents/projects/14_LM1/q2-time/experiments/ray_results/rf/train_rf_4817a_00004_4_bootstrap=False,data_alr_denom_idx=16,data_transform=None,max_depth=16,max_features=0.1000,min_impurity_dec_2023-12-15_09-53-02/model.pkl",127.0.0.1,40336,79.716,262.248,1.26063,1.26063,1.26063,1702630391,1,4817a_00004
train_rf_4817a_00005,2023-12-15_09-53-18,True,"5_bootstrap=False,data_alr_denom_idx=1,data_transform=clr,max_depth=6,max_features=sqrt,min_impurity_decrease=0.0001,min_samples_leaf=0.0000,min_samples_split=0.0001,n_estimators=924",fsbt142-114,1,"/Users/anjaadamov/Documents/projects/14_LM1/q2-time/experiments/ray_results/rf/train_rf_4817a_00005_5_bootstrap=False,data_alr_denom_idx=1,data_transform=clr,max_depth=6,max_features=sqrt,min_impurity_decrease_2023-12-15_09-53-09/model.pkl",127.0.0.1,40347,76.6754,276.643,1.63312,1.63312,1.63312,1702630398,1,4817a_00005
train_rf_4817a_00006,2023-12-15_09-53-24,True,"6_bootstrap=False,data_alr_denom_idx=11,data_transform=ilr,max_depth=21,max_features=0.1000,min_impurity_decrease=0.0010,min_samples_leaf=0.0001,min_samples_split=0.0001,n_estimators=345",fsbt142-114,1,"/Users/anjaadamov/Documents/projects/14_LM1/q2-time/experiments/ray_results/rf/train_rf_4817a_00006_6_bootstrap=False,data_alr_den

(train_rf pid=40375) Train: (48, 21), Test: (27, 21)


2023-12-15 09:53:32,247	INFO tune.py:945 -- Total run time: 61.18 seconds (61.13 seconds for the tuning loop).


{'xgb': ResultGrid<[
   Result(
     metrics={'rmse_train': 357.7010169196166, 'rmse_val': 360.5866496963596, 'done': True, 'trial_id': '646e0_00000', 'experiment_tag': '0_data_alr_denom_idx=11,data_transform=None,eta=0.0500,max_depth=6,min_child_weight=2,subsample=0.8000'},
     path='/Users/anjaadamov/Documents/projects/14_LM1/q2-time/experiments/ray_results/xgb/train_xgb_646e0_00000_0_data_alr_denom_idx=11,data_transform=None,eta=0.0500,max_depth=6,min_child_weight=2,subsample=0.8000_2023-12-15_09-46-13',
     checkpoint=None
   ),
   Result(
     metrics={'rmse_train': 420.1518683701937, 'rmse_val': 406.3351855442469, 'done': True, 'trial_id': '646e0_00001', 'experiment_tag': '1_data_alr_denom_idx=2,data_transform=clr,eta=0.0100,max_depth=6,min_child_weight=3,subsample=0.7000'},
     path='/Users/anjaadamov/Documents/projects/14_LM1/q2-time/experiments/ray_results/xgb/train_xgb_646e0_00001_1_data_alr_denom_idx=2,data_transform=clr,eta=0.0100,max_depth=6,min_child_weight=3,subsample

## Evaluate best models

In [ ]:
class TunedModel:
    def __init__(self, model, data_config):
        self.model = model
        self.data_config = data_config

    def transform(self, data):
        transformed = transform_features(
            data,
            self.data_config["data_transform"],
            self.data_config["data_alr_denom_idx"],
        )
        if isinstance(self.model, xgb.core.Booster):
            return xgb.DMatrix(transformed)
        else:
            return transformed.values

    def predict(self, data):
        transformed = self.transform(data)
        predicted = self.model.predict(transformed)
        return predicted.flatten()

In [ ]:
best_model_dic = {}
for model_type, result_grid in result_dic.items():
    best_model = get_best_model(model_type, result_grid)
    best_data_proc = get_best_data_processing(result_grid)

    best_model_dic[model_type] = TunedModel(best_model, best_data_proc)

best_model_dic

In [ ]:
# ! Perform predictions with best model
def save_predictions(data, tmodel, target, features, split=None):
    # id, true
    saved_pred = data[[target]].copy()
    saved_pred.rename(columns={target: "true"}, inplace=True)
    # pred, split
    saved_pred["pred"] = tmodel.predict(data[features])
    saved_pred["split"] = split
    return saved_pred


non_features = [TARGET, HOST_ID]
features = [x for x in train_val if x not in non_features]

# dic with model_type: df with all predictions within
preds_dic = {}
for model_type, tmodel in best_model_dic.items():
    train_df = save_predictions(train_val, tmodel, TARGET, features, "train")
    test_pred = save_predictions(test, tmodel, TARGET, features, "test")

    preds_dic[model_type] = pd.concat([train_df, test_pred])

In [ ]:
# plot rmse over all experiments # !(OVERALL case)
def _calculate_rmse(pred_df):
    rmse_scores = {}
    for split in pred_df["split"].unique():
        pred_split = pred_df[pred_df["split"] == split].copy()
        rmse = mean_squared_error(
            pred_split["true"].values, pred_split["pred"].values, squared=False
        )
        rmse_scores[split] = rmse
    return rmse_scores


rmse_dic = {}
for model_type, pred_df in preds_dic.items():
    # todo: add a bin_by variable (for "over time" and "over study")
    rmse_dic[model_type] = _calculate_rmse(pred_df)

rmse_df = pd.DataFrame(rmse_dic).T
rmse_df.plot(kind="bar", title="Overall", ylabel="RMSE")

In [ ]:
# plot rmse over true time bin
# ! (TIME case)
model_type = "rf"
pred_df = preds_dic[model_type]
split = None

# bin true columns by months
pred_df["group"] = np.round(pred_df["true"] / DAYS_PER_MONTH, 0).astype(int)

grouped_ser = pred_df.groupby(["group"]).apply(_calculate_rmse)
grouped_df = grouped_ser.apply(pd.Series)
if split is not None:
    grouped_df = grouped_df[[split]].copy()

grouped_df.plot(
    kind="bar", title=f"Model: {model_type}", ylabel="RMSE", figsize=(10, 5)
)

## Compare experiments

In [ ]:
def get_best_model(exp_name, trial_result, metric_ls=["rmse_train", "rmse_val"]):
    # Get the result with the metric and mode defined in tune_config before
    best_result = trial_result.get_best_result()
    config = best_result.config

    # # get config of best model
    # best_result.config

    metrics_ser = best_result.metrics_dataframe[metric_ls].iloc[-1]
    metrics_df = pd.DataFrame({exp_name: metrics_ser})
    return metrics_df, config


def calc_best_metrics(dic_trials):
    df_metrics = pd.DataFrame(index=["rmse_train", "rmse_val"])
    dic_config = {}
    for key, value in dic_trials.items():
        df_best, config = get_best_model(key, value)
        df_metrics = df_metrics.join(df_best)
        dic_config[key] = config

    return df_metrics, pd.DataFrame(dic_config)


def plot_best_metrics(df_metrics):
    df2plot = df_metrics.T.sort_values("rmse_val", ascending=True)
    df2plot.columns = ["train", "validation"]
    # plot settings
    # todo: set default plot settings across package
    plt.style.use("seaborn-v0_8-colorblind")  # ("tableau-colorblind10")
    titlesize = 14
    labelsize = 13
    ticklabel = 12
    plt.rcParams.update({"font.size": labelsize})

    df2plot.plot(kind="bar", figsize=(12, 6))

    plt.xticks(fontsize=ticklabel)
    plt.yticks(fontsize=ticklabel)
    plt.ylabel("RMSE", fontsize=labelsize)
    plt.xlabel("Model type (order: increasing val score)", fontsize=labelsize)
    plt.title("Metrics comparison", fontsize=titlesize)
    plt.show()

In [ ]:
metrics_all, best_configs = calc_best_metrics(
    {
        "xgb": result_dic["xgb"],
        "linreg": result_dic["linreg"],
        "nn": result_dic["nn"],
        "rf": result_dic["rf"],
    }
)
plot_best_metrics(metrics_all)
display(best_configs)

In [ ]:
def _highlight_differing_cols(x):
    """
    Function returning color map of differing columns in x
    Original code used as base:
    https://stackoverflow.com/questions/41654949/pandas-style-function-to-highlight-specific-columns
    """
    # copy df to new - original data is not changed
    df = x.copy()

    # extract list of columns that differ between all models
    ls_col = df.columns[df.nunique(dropna=False) > 1].tolist()

    # select default neutral background
    df.loc[:, :] = "background-color: None"

    # mark columns that differ
    df[ls_col] = "color: red"

    # return colored df
    return df


best_configs.style.apply(_highlight_differing_cols, axis=None)

## Evaluate over training time (example for xgb model here)

In [ ]:
# Get the result with the maximum test set `mean_accuracy`
best_xgb = result_dic["xgb"].get_best_result()
best_xgb.metrics_dataframe.plot("training_iteration", ["rmse_train", "rmse_val"])

In [ ]:
ax = None
for result in result_dic["xgb"]:
    label = f"data_transform={result.config['data_transform']}, \
        max_depth={result.config['max_depth']}"
    if ax is None:
        ax = result.metrics_dataframe.plot(
            "training_iteration", "rmse_val", label=label
        )
    else:
        result.metrics_dataframe.plot(
            "training_iteration", "rmse_val", ax=ax, label=label
        )
ax.legend(bbox_to_anchor=(1.1, 1.05))

ax.set_title("rsme_val vs. training iteration for all trials")
ax.set_ylabel("RMSE_val")